In [9]:
import os
import requests
import pandas as pd
import numpy as np
print('kjsbf;ab')
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto
import torch
import time
import pandas as pd, html
from user_info import get_user_info
import requests
from tqdm.notebook import tqdm
from user_info import get_user_data_search_embed
print('import')
from job_info import fetch_jobs
from tqdm import tqdm
from vllm import SamplingParams, LLM
print('imports done')
sampling_params = SamplingParams(temperature=0, max_tokens=100)
llm = LLM(model="mistralai/Mistral-7B-Instruct-v0.2")

dataset = pd.read_csv('/home/ubuntu/infoedge/llama-recipes/examples/relevant_jobs_classification/data/analysis_dataset.csv')
import gradio as gr

kjsbf;ab
import
imports done
INFO 02-27 18:33:56 llm_engine.py:73] Initializing an LLM engine with config: model='mistralai/Mistral-7B-Instruct-v0.2', tokenizer='mistralai/Mistral-7B-Instruct-v0.2', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, enforce_eager=False, seed=0)
INFO 02-27 18:34:11 llm_engine.py:223] # GPU blocks: 922, # CPU blocks: 2048
INFO 02-27 18:34:14 model_runner.py:394] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 02-27 18:34:20 model_runner.py:437] Graph capturing finished in 6 secs.


In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv('/home/ubuntu/infoedge/llama-recipes/examples/relevant_jobs_classification/data/analysis_dataset.csv')


In [4]:
dataset.sample(5)

,job_id,application_id,user_id
76042,1374952,144331588,533388
28540,1374193,144199195,563840
113904,1373894,144401131,2807486
93907,1374926,144365860,2719861
42607,1374538,144230269,2336526


In [8]:
dataset.loc[dataset['job_id']==1374926]

,job_id,application_id,user_id
59814,1374926,144327300,1050531
59815,1374926,144324432,1933640
59816,1374926,144307403,2732972
59817,1374926,144293816,2799756
59818,1374926,144292683,2739316
59819,1374926,144292572,1009440
59820,1374926,144292482,2700661
59821,1374926,144290572,2161231
59822,1374926,144288460,1153685
59823,1374926,144285583,1701655


In [10]:
def fetch_job_title(job_id):
    
    '''
    Returns Job Data[Hirst, Source : ElasticSearch] for the last_k days, and with flexible page size (<10,000)
    
    Input : Last-K Days, and Page result size
    
    Output : Pandas Dataframe with jobs from the last k days 
    '''
    
    # columns = ['id', 'title','companyStatus', 'min', 'max', 'premium', 'applyCount', 'locations','tags','brandJobFlag']     #Added column name for companyStatus
    # new_cols = ['jobid', 'title','companyStatus', 'min', 'max', 'premium', 'applyCount', 'locations','tags', 'brandJobFlag']       #Added column name for companyStatus
    
    url = "http://internal-java-job-searcher-email-backend-1607736061.ap-south-1.elb.amazonaws.com/v1/user/-777/job/jobcode?query=<JOBID>"
    url = url.replace('<JOBID>',job_id)
    payload={}
    headers = {}
    
    response = requests.request("GET", url, headers=headers, data=payload)
    payload={}
    headers = {}
    
    response = requests.request("GET", url, headers=headers, data=payload)
    job_data = response.json()

    title = job_data[0]['title']
    jd = job_data[0]['introText']
    jd = html.unescape(jd)

    return title + '\n' + jd

In [11]:
# t = fetch_job_title(str(1373922))

In [12]:
# t

In [13]:
def get_answer_from_vllm(resume_text, job_title, llm):

    #define the input prompt here
    input_prompt = f'''<s>[INST]You are an accurate AI agent working for a job platform. You will be given the raw 
                            unstructured text of a user's resume, and the description of a job that they have applied to. Your task 
                            is to check if the resume contains prior work experience relevant to the job title being provided.
                            Respond with a JUST A SINGLE WORD answer "yes" or "no". If there's any ambiguity, answer with "no"
                            This is the job description {{job_title}}\n
                            This is the resume text:\n{{resume_text}}\n
                            The answer is: [\INST]
                            '''

    input_prompt = input_prompt.format(resume_text=resume_text,
                   job_title=job_title)
    
    outputs = llm.generate(input_prompt, sampling_params)
    generated_text = outputs[0].outputs[0].text

    return generated_text

In [14]:
def make_clickable(link):
    return f'<a href="{link}" target="_blank">{link}</a>'

In [15]:
def run_predictions_for_job(job_id):
    job_id=int(job_id)
    print(job_id)
    responses = {}
    ds = dataset.loc[dataset['job_id']==job_id]
    ds = ds.head(5)
    ex = ds[['job_id','user_id']].to_numpy().tolist()
    # print('ye bhi',ex)
    for jobid, userid in tqdm(ex):
        print(jobid)

        user_id_list = dataset.loc[dataset['job_id']==jobid]['user_id'].tolist()
        try:
            jt = fetch_job_title(str(jobid))
            # print(jt)
        except Exception as e:
            print(f'Error while getting job data for job id {jobid} is {e}\n')
            continue

        if jobid in responses:
            pass
        else:
            responses[jobid] = {}

        try:
            es_output = get_user_info(userid)
            rt = es_output['resume'][0]
            rt = html.unescape(rt)
        except Exception as e:
            print(f'Error while getting resume text for user_id {userid} is {e}\n')
            continue
        
        full_doc = get_answer_from_vllm(rt,jt, llm)
        print('ye kra',full_doc)
        responses[jobid][userid] = full_doc

    # return responses

    # print(responses)

        # df = get_user_data_search_embed(ds['user_id'].tolist())
    print(type(job_id))
    new_df = pd.DataFrame(responses[int(job_id)],index=[0]).T.reset_index()
    new_df.columns=['id','info']
    df = get_user_data_search_embed(new_df['id'].tolist())
        
    final_usr=pd.DataFrame()

    for x in df['id']:
        hy = "https://search.iimjobs.com/profile/userid"
        mini = df.loc[df['id'] == x].copy()
        mini['user_profile'] = [hy.replace("userid", str(x))]
        final_usr = pd.concat([final_usr, mini], ignore_index=True)

    final_usr.reset_index(drop=True, inplace=True)
    final_usr['user_profile'] = final_usr['user_profile'].apply(lambda x: make_clickable(x))

    final_usr = final_usr[['id','current_designation','user_experience','user_profile']]
    final_usr['id']=final_usr['id'].astype(int)

    fd = pd.merge(final_usr,new_df,on='id')
    # fd['user_profile'] = fd['user_profile'].apply(lambda x: make_clickable(x))


    return fd


In [16]:
# df = run_predictions_for_job(1373922)

In [17]:
# df[['info']].values

In [18]:
# df

In [19]:
import pandas as pd

def split_yes_no_extra_info(text):
    text_lower = text.lower()
    if 'yes' in text_lower:
        return 'yes', text.replace('yes', '').strip()
    elif 'no' in text_lower:
        return 'no', text.replace('no', '').strip()
    else:
        return None, text.strip()

# Assuming your DataFrame is named df and the column is named "info"
# Replace 'your_column_name' with the actual column name in your DataFrame

# Create new columns for 'yes_no' and 'extra_info'
# df['yes_no'], df['extra_info'] = zip(*df['info'].apply(split_yes_no_extra_info))

# # Display the resulting DataFrame
# print(df[['yes_no', 'extra_info']])


In [20]:
# df['extra_info'] = df['extra_info'].replace('\n', ' ', regex=True)
# df['extra_info'] = df['extra_info'].replace('\n', ' ', regex=True).str.strip()

# # Remove extra spaces
# df['extra_info'] = df['extra_info'].replace('\s+', ' ', regex=True)


In [21]:
# df

In [23]:
# df

In [24]:
# d.drop(['yes_no','extra_info'],axis=1,inplace=True)

In [26]:
import gradio as gr

iface = gr.Interface(
    fn=run_predictions_for_job,
    inputs=[gr.Textbox(label="JOB ID", placeholder="Enter Job ID", style={"height": "100px", "width": "400px"})],
    outputs=[gr.HTML(label="RESULTS", style={"height": "200px", "width": "80%"})]
)


iface.launch('0.0.0.0', share=True)

# if __name__ == "__main__":
#     demo.launch(share=True)

TypeError: Textbox.__init__() got an unexpected keyword argument 'style'

In [14]:
import configparser
import pymysql
import pandas as pd

In [15]:
config=configparser.ConfigParser()
config.read('/home/ubuntu/infoedge/llama-recipes/examples/configs/config.ini')

['/home/ubuntu/infoedge/llama-recipes/examples/configs/config.ini']

In [16]:
mydb= pymysql.connect(
host=config['iimjobs_db']['hostname'],
user=config['iimjobs_db']['username'],
password=config['iimjobs_db']['password'],
db=config['iimjobs_db']['db_name'])

mycursor = mydb.cursor()

In [17]:

query = "select jobid,userid from usr_applied_jobs_info where jobid=1373922"

mycursor.execute(query)
result = mycursor.fetchall()

In [18]:
def job_application(jobid):
    
    query = "select jobid,userid from usr_applied_jobs_info where jobid=JOBID"
    query = query.replace('JOBID',str(jobid))
    print(query)
    mycursor.execute(query)
    result = mycursor.fetchall()
    return pd.DataFrame(result,columns=['job_id','user_id'])

In [19]:
result=job_application(1373922)

select jobid,userid from usr_applied_jobs_info where jobid=1373922


In [20]:
result

,job_id,user_id
0,1373922,237375
1,1373922,911674
2,1373922,397888
3,1373922,179817
4,1373922,2479334
...,...,...
69,1373922,1033614
70,1373922,153958
71,1373922,2804994
72,1373922,2023317
